In [97]:
import pandas as pd
import json
from pprint import pprint

In [133]:
ms = pd.read_csv('datasets/MassShootingsComplete-counties.csv')
pop = pd.read_csv('datasets/CountyPopulationAllYears.csv')

In [ ]:
def transforms(name: str) -> str:
    if 'Clarke' in name: return 'Clarke County'
    if 'Washington' in name: return 'District of Columbia'
    name = name.replace('Saint', 'St.').replace('Nashville-', '').replace('Vista', 'Buena Vista').replace('Compton', 'Los Angeles County')
    name = name.replace('Monterey Park', 'Monterey County')
    return name.strip()


def try_find(name, state) -> str | None:

    if name in pop[pop['STNAME']==state]['CTYNAME'].values: return name

    if transforms(name) in pop[pop['STNAME']==state]['CTYNAME'].values: return transforms(name)

    #try county
    n = transforms(name)
    if 'County' not in n: n += ' County'
    if n in pop[pop['STNAME']==state]['CTYNAME'].values: return n

    #try parish
    n = transforms(name)
    if 'Parish' not in n: n += ' Parish'
    if n in pop[pop['STNAME']==state]['CTYNAME'].values: return n

    #try city
    n = transforms(name)
    if 'city' not in n: n += ' city'
    if n in pop[pop['STNAME']==state]['CTYNAME'].values: return n

    #try Municipality
    n = transforms(name)
    if 'city' not in n: n += ' Municipality'
    if n in pop[pop['STNAME']==state]['CTYNAME'].values: return n
    
    else: return None


def extract_county(row) -> str:
    '''try:'''
    json_data = row['json']
    state = row['State'].strip()

    places = json.loads(json_data)
    if places:
        names = places[0].get("display_name", "")
        # Cerca "County", "Parish", o altres entitats equivalents en display_name
        
        delims = [",", ")", "("]

        for delim in delims:
            names = "_".join(names.split(delim))
        names = names.split("_")
        names = [n.strip() for n in names if n != '']

        for n in names:
            cname = try_find(n, state)
            if cname: return cname
            else: continue
    
        return None
    '''except (json.JSONDecodeError, IndexError, KeyError):
        return None'''

In [ ]:
count = 0

for i in range(len(ms)):
    row = ms.loc[i]
    js = row['json']
    state = row['State']
    if js == '[]': count += 1
    elif not extract_county(row):
        names = json.loads(js)[0].get("display_name", "")
        count += 1

print(count)

35


In [144]:
ms['county'] = ms.apply(extract_county, axis=1)
ms[ms['county'].isnull()]     # només els que tenen json Null !!!! big success!

,Incident ID,Incident Date,State,City Or County,json,county,Address,Victims Killed,Victims Injured,Suspects Killed,Suspects Injured,Suspects Arrested
64,2988723,2024-08-06T00:00:00Z,West Virginia,Hedgesville (Cherry Run),[],None,Householder Rd,0,4,0,0,1
192,2934048,2024-06-07T00:00:00Z,Missouri,Saint Louis (Jennings),[],None,2600 block of Shannon Ave,1,4,0,0,0
464,2747153,2023-11-04T00:00:00Z,Ohio,Cleveland (Richmond Heights),[],None,500 block of Towne Ln,1,3,0,0,1
566,2701233,2023-09-06T00:00:00Z,Texas,El Paso (Socorro),[],None,253 Flor Margarita Blvd,0,4,0,0,3
853,2592771,2023-05-06T00:00:00Z,Mississippi,Robinsonville (Tunica Resorts),[],None,2669 Kirby Rd,0,4,0,0,0
1019,2508683,2023-01-22T00:00:00Z,Mississippi,Robinsonville (Tunica Resorts),[],None,1010 Casino Center Dr,0,4,0,0,1
1030,2502550,2023-01-14T00:00:00Z,Missouri,Saint Louis (Moline Acres),[],None,Berwyn Dr and Lanier Dr,0,4,0,0,0
1252,2400998,2022-08-29T00:00:00Z,Missouri,Saint Louis (Jennings),[],None,8800 block of Cozens Ave,0,4,0,0,0
1456,2320673,2022-06-04T00:00:00Z,Texas,El Paso (Socorro),[],None,10851 Thunder Rd,0,5,0,0,0
1772,2163772,2021-11-09T00:00:00Z,Louisiana,Hahnville (Killona),[],None,160 Killona Dr,1,4,0,0,1


In [ ]:
def get_fips(row) -> str:
    """
    Retorna el valor de 'fips' per una fila de df1 basant-se en les columnes 'State' i 'county'.
    
    :param row: Una fila de df1 (que conté 'State' i 'county').
    :param pop: DataFrame que conté les columnes 'STNAME', 'CTYNAME' i 'FIPS'.
    :return: El valor de 'fips' corresponent.
    """
    # Comprovar si 'county' no és None abans d'aplicar strip()
    county_value = row['county']
    if county_value is not None:
        county_value = county_value.strip()  # eliminar espais blancs si no és None
    else:
        county_value = ""  # Si és None, assignar una cadena buida per evitar errors
    
    # Filtrar df2 per obtenir la fila amb la mateixa combinació de 'State' i 'county'
    fips_value = pop[(pop['STNAME'] == row['State']) & (pop['CTYNAME'] == county_value)]['FIPS']
    
    # Retornar el primer valor (ja que hauria de ser únic)
    return f'{fips_value.iloc[0]:02d}' if not fips_value.empty else None


In [160]:
ms['FIPS'] = ms.apply(get_fips, axis=1)

In [ ]:

len(ms[ms['State'].isin(pop['STNAME'].values)]) == len(ms)  #perfecte!

True

In [149]:
print(len(ms[ms['FIPS'].isnull()]))
ms[ms['FIPS'].isnull()]

35


,Incident ID,Incident Date,State,City Or County,json,county,Address,Victims Killed,Victims Injured,Suspects Killed,Suspects Injured,Suspects Arrested,FIPS
64,2988723,2024-08-06T00:00:00Z,West Virginia,Hedgesville (Cherry Run),[],None,Householder Rd,0,4,0,0,1,None
192,2934048,2024-06-07T00:00:00Z,Missouri,Saint Louis (Jennings),[],None,2600 block of Shannon Ave,1,4,0,0,0,None
464,2747153,2023-11-04T00:00:00Z,Ohio,Cleveland (Richmond Heights),[],None,500 block of Towne Ln,1,3,0,0,1,None
566,2701233,2023-09-06T00:00:00Z,Texas,El Paso (Socorro),[],None,253 Flor Margarita Blvd,0,4,0,0,3,None
853,2592771,2023-05-06T00:00:00Z,Mississippi,Robinsonville (Tunica Resorts),[],None,2669 Kirby Rd,0,4,0,0,0,None
1019,2508683,2023-01-22T00:00:00Z,Mississippi,Robinsonville (Tunica Resorts),[],None,1010 Casino Center Dr,0,4,0,0,1,None
1030,2502550,2023-01-14T00:00:00Z,Missouri,Saint Louis (Moline Acres),[],None,Berwyn Dr and Lanier Dr,0,4,0,0,0,None
1252,2400998,2022-08-29T00:00:00Z,Missouri,Saint Louis (Jennings),[],None,8800 block of Cozens Ave,0,4,0,0,0,None
1456,2320673,2022-06-04T00:00:00Z,Texas,El Paso (Socorro),[],None,10851 Thunder Rd,0,5,0,0,0,None
1772,2163772,2021-11-09T00:00:00Z,Louisiana,Hahnville (Killona),[],None,160 Killona Dr,1,4,0,0,1,None


In [154]:
# remove "null" rows...
ms = ms.dropna(subset=['FIPS'])
print(len(ms[ms['FIPS'].isnull()]))

0


In [155]:
ms.to_csv('datasets/MashShootingsComplete_FIPS.csv', index=False)

In [164]:
si = pd.read_csv('datasets/Schoolincidents-json.csv')

In [165]:
si['county'] = si.apply(extract_county, axis=1)

In [169]:
print(len(si[si['county'].isnull()]), len(si))

23 2000


In [176]:
si['FIPS'] = si.apply(get_fips, axis=1).astype(str)

In [177]:
for i in range(len(si)): print(si['FIPS'].loc[i])

24027
1089
53063
45019
51570
6037
22033
22033
48485
18141
47065
22033
37109
39035
None
39035
17031
16027
28035
35001
48257
39035
40109
1101
18097
37067
12009
48215
32003
26099
37183
48001
28087
6097
16047
22125
25023
35001
48309
17099
37157
48441
48061
51179
4025
21111
21111
36029
36047
22055
48141
55079
36047
16005
56037
35061
27053
26031
13077
53005
22105
36047
53033
42081
47009
47157
10003
37119
17097
37061
36047
53011
1095
47157
None
20203
12031
4019
36037
48053
46099
41051
35001
54039
36061
39061
22011
22055
6077
47037
12055
46093
None
46093
48029
48225
36081
13121
22073
24017
39049
18003
48041
36085
22063
39113
21139
21173
27123
48439
35001
29189
12057
48141
41029
39049
26125
51137
13217
39061
36029
47065
22071
6067
13121
12095
39101
13135
13077
42045
36055
45059
13121
12073
12057
29077
40145
17043
48451
45067
45023
47069
21111
13055
37063
18095
46099
54053
39061
12117
53033
1073
12019
51710
51700
45003
31055
47157
17043
19057
42101
37183
37097
53011
37183
28117
12086
31111
39061